In [14]:
print("general libs to download. Use ChatGPT and pip install to handle issues")

#https://huggingface.co/hkunlp/instructor-large
#pip install InstructorEmbedding

import re
import json

from bs4 import BeautifulSoup

import json
from InstructorEmbedding import INSTRUCTOR

import torch
from transformers import BartForConditionalGeneration, BartTokenizer




general libs to download. Use ChatGPT and pip install to handle issues


In [1]:
print("Embedding test")

from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')

sentence = "Hello AI world"
instruction = "hello"

embeddings = model.encode([[instruction,sentence]])
print(embeddings[0])


Embedding test


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
[-5.67759424e-02 -7.85741303e-03 -1.55622931e-02  2.27733888e-03
  3.52524333e-02  4.26125303e-02  1.45384949e-02 -2.61808876e-02
 -8.55610706e-03  3.41044590e-02  5.80871291e-02  2.10360810e-02
  9.20202397e-03  5.33556864e-02 -3.78961414e-02  7.10365037e-03
 -4.81327996e-02 -1.68315833e-03 -1.69199761e-02 -1.54205607e-02
  4.21466678e-02 -1.86046283e-03 -1.15041248e-05  3.93789783e-02
 -7.92815629e-03  4.13183197e-02 -3.82487252e-02  1.19009325e-02
  5.91590218e-02 -7.17932507e-02  3.91817512e-03 -3.09534799e-02
 -2.10253596e-02 -2.01134663e-02 -1.52507117e-02  1.25192972e-02
  2.95347646e-02  3.83191034e-02  1.45295914e-02  3.14703584e-02
 -1.17491484e-02 -1.88130196e-02  2.67630108e-02 -2.75423471e-02
  4.31421809e-02  6.73846994e-03 -2.47243270e-02 -1.85277294e-02
  7.30894320e-03 -4.28918144e-03 -4.82225716e-02 -5.07485643e-02
 -1.63041651e-02  2.46827584e-02 -1.86329416e-03 -2.11289655e-02
 -6.05235398e-02 -2.29059937e-04  6.556206

In [6]:
# Quadrant quick start man pages: https://qdrant.tech/documentation/quick-start/
## create embaddings
import json
from qdrant_client.http.models import PointStruct
from qdrant_client import QdrantClient

instruction = "hello"

data = ["Hello AI world", "Bye AI world", "Buy me some pizza", "The lection is very good", "Nice to meet you"]

dataPoints = []
id = 0
for sentence in data:
    id += 1
    embeddings = model.encode([[instruction,sentence]])
    record = {"id": id, "emb": embeddings[0].tolist(), "payload": {"txt": sentence}}
    dataPoints.append(
        PointStruct(id=id, vector=embeddings[0], payload=record)
    )

from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="hello",
    vectors_config=VectorParams(size=768, distance=Distance.DOT),
)


operation_info = client.upsert(
    collection_name="hello",
    wait=True,
    points=dataPoints,
)



In [9]:
# Test questions
search_qq = ["how do you do"]
search_exp_id = 1
search_exp_hits = 0

for search_one_q in search_qq:
    search_embd = model.encode([[instruction,search_one_q]])
    search_result = client.search(
        collection_name="hello", query_vector=search_embd[0], limit=2
    )


    for res in search_result:
        print(str(res.id) + ": " + str(res.score) + ": " + str(res.payload['payload']))



5: 0.9094763994216919: {'txt': 'Nice to meet you'}
1: 0.8890741467475891: {'txt': 'Hello AI world'}


In [10]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')



In [11]:
input_sentence = "How do I update my user name or password?"

batch = tokenizer(input_sentence, return_tensors='pt')
generated_ids = model.generate(batch['input_ids'])
generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_sentence)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['How do I reset my password?']
